## RAPADA PARTNERS - CASE STUDY - Madrid´s Real Estate Market

In [ ]:
#Import libraries

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.ticker as ticker
import folium
from folium.plugins import MarkerCluster
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, r2_score



In [ ]:
# Import Dataset
url = "technical_interview.parquet"
df = pd.read_parquet(url)
df.head()

In [ ]:
df_original = df.copy()


# 1. Initial data exploration, cleaning and handling missing or anomalous values
##### Before performing any in depth analysis, we begin by understandig de dataset structure and addresing missing values or outliers that could affect our results

In [ ]:
print('----------------------------------------------------------------------------------------------------')
print('Dataset Shape (rows, columns):')
print(df.shape)

print('\n ----------------------------------------------------------------------------------------------------')
print('Column Names')
print(df.columns)

print('\n ----------------------------------------------------------------------------------------------------')
print('Data Types and non-null counts')
print(df.info())

print('\n ----------------------------------------------------------------------------------------------------')
print('Missing Values per column (sorted in descending order):')
missing_values = df.isnull().sum().sort_values(ascending=False)
print(missing_values[missing_values > 0])


print('\n ----------------------------------------------------------------------------------------------------')
print('\n General Statistics (numerical variables)')
print(df.describe().T)

In [ ]:
df["is_exterior"] = df["is_exterior"].astype(bool)

In [ ]:
# Summary Table including types and nulls
summary_table = pd.DataFrame(df.dtypes, columns=['Type']).T

missing_count = pd.DataFrame(df.isnull().sum()).T
missing_count.index = ['Missing Values (nb)']

missing_percentage = pd.DataFrame(df.isnull().mean() * 100).T
missing_percentage.index = ['Missing Values (%)']


summary_table = pd.concat([summary_table, missing_count, missing_percentage])

pd.set_option('display.max_columns', None)
print('Summary Table')
display(summary_table)


In [ ]:
# Unique values and counts for location-related columns
print("\n Unique values in 'state':")
print(f"{df['state'].nunique()} unique values -> {df['state'].unique()}")
print("Counts:\n", "Column:", df['state'].value_counts())

print("\n Unique values in 'province':")
print(f"{df['province'].nunique()} unique values -> {df['province'].unique()}")
print("Counts:\n", "Column:", df['province'].value_counts())

print("\n Unique values in 'country_code':")
print(f"{df['country_code'].nunique()} unique value -> {df['country_code'].unique()}")
print("Counts:\n", "Column:", df['country_code'].value_counts())

# Since the goal is to analyze the Madrid real estate market, we filter out properties outside Community of Madrid, keeping only those within the province of Madrid.


### 1.1 Missing values

In [ ]:
missing_values = pd.DataFrame({
    "Missing values": df.isnull().sum(),
    "Missing (%)": df.isnull().mean() * 100,
    "Data type": df.dtypes
})

missing_values = missing_values[missing_values["Missing values"] > 0].sort_values("Missing (%)", ascending=False)
display(missing_values)

In [ ]:
# Check if the missing values of n_rooms and n_baths it´s related to the property_type

##############################ROOMS###############################################
rooms = df[df["n_rooms"].isnull()]
print(f"Number of properties with missing n_rooms: {rooms.shape[0]}")
print("\nProperty types with missing n_rooms:")
print(rooms["property_type"].value_counts())
##############################BATHS###############################################
baths = df[df["n_baths"].isnull()]
print(f"\nNumber of properties with missing n_baths: {baths.shape[0]}")
print("\nProperty types with missing n_baths:")
print(baths["property_type"].value_counts())

# Conclusion: missing values of rooms and baths were mostly found in non-residential properties where these features are irrelevant.
rooms_baths = ["n_rooms", "n_baths"]
df[rooms_baths] = df[rooms_baths].fillna(0)


In [ ]:
area_price = ["area", "price"]
df.dropna(subset=area_price, inplace=True)

boolean_columns = [
    "has_elevator", "has_terrace", "is_exterior", "has_swimming_pool",
    "has_parking", "has_garden", "has_energy_certificate", "has_floorplan",
    "has_virtual_tour", "approximate_location"
]
df[boolean_columns] = df[boolean_columns].fillna(False)


categorical_columns = ["property_state", "district", "quarter", "province", "postcode"]
df[categorical_columns] = df[categorical_columns].fillna("Unknown")

print("Remaining missing values after cleaning:")
print(df.isnull().sum()[df.isnull().sum() > 0])


In [ ]:
floor_missing = df[df["floor"].isnull()]

print(f"Total properties with missing 'floor': {floor_missing.shape[0]}\n")
print("Property types with missing 'floor':")
print(floor_missing["property_type"].value_counts())

# Checking whether the missing values make sense before making conclusions
keywords = ["chalet", "adosado", "casa"]
for word in keywords:
    count = floor_missing["ad_description"].str.contains(word, case=False, na=False).sum()
    print(f"Occurrences of '{word}' in ad_description: {count}")


df["floor"] = df["floor"].replace(0, np.nan)

df["has_floor_info"] = df["floor"].notnull().astype(int)

### 1.2 Negative values

In [ ]:
negative_values = df[
    (df["price"] <= 0) |
    (df["area"] <= 0) |
    (df["n_rooms"] < 0) |
    (df["n_baths"] < 0) |
    (df["floor"] < 0) |
    (df["price_down_from"] < 0)
]

# We will handle this values following:
#  price and area <= 0 → Drop values, probably invalid data 
df = df[(df["price"] > 0) & (df["area"] > 0)]

#  n_rooms and n_baths < 0 → Set to 0 probably missing/incorrect data
df.loc[df["n_rooms"] < 0, "n_rooms"] = None
df.loc[df["n_baths"] < 0, "n_baths"] = None

#  floor < 0 → Set to -1 (we will use -1 for "not applicable" floor)
df.loc[df["floor"] < 0, "floor"] = 0

#  price_down_from < 0 → Set to NaN 
df.loc[df["price_down_from"] < 0, "price_down_from"] = None

print(f"Remaining records after cleaning: {df.shape[0]}")


### After this analysis we reduce the dataset to include only the relevant information

In [ ]:

final_columns = [
    "property_type", "transaction_type", "n_rooms", "n_baths", "area", "floor","has_elevator", "has_terrace", "is_exterior",
    "has_swimming_pool","has_parking", "has_garden","energy_certificate_consumption", "has_energy_certificate",
    "has_floorplan", "property_state","latitude", "longitude" , "province", "city","district","quarter","price_down_from", "price"
]
df= df[final_columns]
df


In [ ]:
# Summary Table including types and nulls
summary_table = pd.DataFrame(df.dtypes, columns=['Type']).T

missing_count = pd.DataFrame(df.isnull().sum()).T
missing_count.index = ['Missing Values (nb)']

missing_percentage = pd.DataFrame(df.isnull().mean() * 100).T
missing_percentage.index = ['Missing Values (%)']


summary_table = pd.concat([summary_table, missing_count, missing_percentage])

pd.set_option('display.max_columns', None)
print('Summary Table')
display(summary_table)

df["is_exterior"] = df["is_exterior"].astype(bool)

In [ ]:
for col in df.columns:
    print(f"\ Column: {col}")
    print(df[col].value_counts(dropna=False)) 
    print("-" * 50)


### 1.3 Outliers
For the porpuse of the exploratory analysis, we will remove extreme outliers in price and area to avoid distorsions in following visualizations and summary statistics. However we will keep a copy of the original dataset for later modeling

In [ ]:
print("Price summary:")
print(df["price"].describe())
print("\nArea summary:")
print(df["area"].describe())
print("\nPrice per m² summary:")
df["price_per_m2"] = df["price"] / df["area"]
print(df["price_per_m2"].describe())

# Visualizing the distribution before cleaning
fig, axs = plt.subplots(1, 3, figsize=(18, 5))
sns.boxplot(x=df["price"], ax=axs[0])
axs[0].set_title("Boxplot of price")
sns.boxplot(x=df["area"], ax=axs[1])
axs[1].set_title("Boxplot of area")
sns.boxplot(x=df["price_per_m2"], ax=axs[2])
axs[2].set_title("Boxplot of price per m²")
plt.show()

df_original = df.copy()

# Remove outliers using the IQR Method
def remove_outliers(df, column):
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1
    lower = Q1 - 1.5 * IQR
    upper = Q3 + 1.5 * IQR
    print(f"Filtering outliers in {column}: keeping values between {lower:.2f} and {upper:.2f}")
    return df[(df[column] >= lower) & (df[column] <= upper)]

df = remove_outliers(df, "price")
df = remove_outliers(df, "area")
df = remove_outliers(df, "price_per_m2")

# Visualizing the distribution after cleaning
fig, axs = plt.subplots(1, 3, figsize=(18, 5))
sns.boxplot(x=df["price"], ax=axs[0])
axs[0].set_title("Boxplot of price (cleaned)")
sns.boxplot(x=df["area"], ax=axs[1])
axs[1].set_title("Boxplot of area (cleaned)")
sns.boxplot(x=df["price_per_m2"], ax=axs[2])
axs[2].set_title("Boxplot of price per m² (cleaned)")
plt.show()


In [ ]:
## NUMERICAL VALUES
num_cols = df.select_dtypes(include='number').columns
df[num_cols].describe().T

In [ ]:

fig, axes = plt.subplots(1, 3, figsize=(20, 5))

# Price Histogram, to visualize the price distribution
sns.histplot(df['price'], kde=True, bins=30, color='blue', ax=axes[0])
axes[0].set_title('Price Distribution')
axes[0].set_xlabel('Price (€)')
axes[0].set_ylabel('Frequency')

# Are Histogram, to visualize the area distribution
sns.histplot(df['area'], kde=True, bins=30, color='orange', ax=axes[1])
axes[1].set_title('Area Distribution')
axes[1].set_xlabel('Área (m²)')
axes[1].set_ylabel('Frequency')

df['price_per_m2'] = df['price'] / df['area']

# Price per m2 distribution
sns.histplot(df['price_per_m2'], kde=True, bins=30, color='green', ax=axes[2])
axes[2].set_title('Price per m2 Distribution')
axes[2].set_xlabel('Price per m² (€)')
axes[2].set_ylabel('Frequency')

plt.tight_layout()
plt.show()


In [ ]:
# Numerical columns
num_cols = df.select_dtypes(include=["int", "float"]).columns

# Correlation matrix
corr_matrix = df[num_cols].corr()

# heatmap
plt.figure(figsize=(10, 8))
sns.heatmap(corr_matrix, annot=True, fmt=".2f", cmap="coolwarm", square=True)
plt.title("Correlation matrix (numerical variables)")
plt.show()

sorted_corr = corr_matrix.abs().unstack().sort_values(ascending=False)
sorted_corr = sorted_corr[(sorted_corr < 1.0)]

# Strong correlations 
strong_corr = sorted_corr[sorted_corr >= 0.7]
print("Strong correlations (rate ≥ 0.7):\n", strong_corr)

# Medium correlations (rate between 0.3 and 0.7)
medium_corr = sorted_corr[(sorted_corr >= 0.3) & (sorted_corr < 0.7)]
print("\n Medium correlations (0.3 ≤ rate < 0.7):\n", medium_corr)



In [ ]:
district_price = df.groupby("district")["price"].mean().sort_values(ascending=False)

top_districts = district_price.head(15)
bottom_districts = district_price.tail(15)

fig, axes = plt.subplots(1, 2, figsize=(15, 8), sharex=False)

sns.barplot(x=top_districts.values, y=top_districts.index,hue=top_districts.index, palette="flare", ax=axes[0])
axes[0].set_title("Top 20 Most Expensive Districts", fontsize=14)
axes[0].set_xlabel("Average price(€)")
axes[0].set_ylabel("District")
axes[0].xaxis.set_major_formatter(ticker.FuncFormatter(lambda x, _: f"{int(x):,}".replace(",", ".")))

sns.barplot(x=bottom_districts.values, y=bottom_districts.index,hue=bottom_districts, palette="crest", ax=axes[1])
axes[1].set_title("Top 20 Least Expensive Districts", fontsize=14)
axes[1].set_xlabel("Average price(€)")
axes[1].set_ylabel("District")
axes[1].xaxis.set_major_formatter(ticker.FuncFormatter(lambda x, _: f"{int(x):,}".replace(",", ".")))

plt.suptitle("Comparison of Average Property Prices by District", fontsize=16)
plt.tight_layout()
plt.show()


In [ ]:
df["price_per_m2"] = df["price"] / df["area"]

district_price = df.groupby("district")["price_per_m2"].mean().sort_values(ascending=False)


top_districts = district_price.head(15)
bottom_districts = district_price.tail(15)

fig, axes = plt.subplots(1, 2, figsize=(15, 8), sharex=False)

sns.barplot(x=top_districts.values, y=top_districts.index,hue=top_districts.index, palette="flare", ax=axes[0])
axes[0].set_title("Top 20 Most Expensive Districts", fontsize=14)
axes[0].set_xlabel("Average price per m2(€)")
axes[0].set_ylabel("District")
axes[0].xaxis.set_major_formatter(ticker.FuncFormatter(lambda x, _: f"{int(x):,}".replace(",", ".")))

sns.barplot(x=bottom_districts.values, y=bottom_districts.index,hue=bottom_districts, palette="crest", ax=axes[1])
axes[1].set_title("Top 20 Least Expensive Districts", fontsize=14)
axes[1].set_xlabel("Average price per m2(€)")
axes[1].set_ylabel("")
axes[1].xaxis.set_major_formatter(ticker.FuncFormatter(lambda x, _: f"{int(x):,}".replace(",", ".")))

plt.suptitle("Comparison of Average Property Prices per m2 by District", fontsize=16)
plt.tight_layout()
plt.show()


In [ ]:
# Create a base map centered on Madrid
madrid_map = folium.Map(location=[40.4168, -3.7038], zoom_start=11, tiles="CartoDB positron")

# Create a marker cluster to group nearby points
marker_cluster = MarkerCluster().add_to(madrid_map)

for idx, row in df.iterrows():
    folium.CircleMarker(
        location=[row["latitude"], row["longitude"]],
        radius=3,
        color="blue",
        fill=True,
        fill_color="blue",
        fill_opacity=0.5,
        popup=f"Price: €{row['price']:,}\nType: {row['property_type']}\nDistrict: {row['district']}"
    ).add_to(marker_cluster)


madrid_map



"""
# Modeling Approach: separating Sales and Rentals

To build accurate predictive models, we separate the dataset into **sales** (transaction_type = 'buy' or 'venta') and **rentals** (transaction_type = 'rent' or 'alquiler'). 
This distinction is important because the dynamics of these two markets behave differently: 
- Properties for sale and those for rent have different price ranges and influencing factors.
- Mixing them in a single model could lead to poor predictions and misleading insights.

We will train two independent models:
1. **Sales Model**: Predicts the market price of properties for sale, allowing to detect underpriced investment opportunities.
2. **Rentals Model**: Predicts rental prices and helps estimate potential gross rental yields to identify high-return areas.

This separation ensures that the analysis captures the unique characteristics of each market and provides actionable insights for our investors, Rapada Partners.

We will use a Random Forest Regressor as our predictive model:
1. **Non-linear relationships and interaction effects.**: Random Forest can handle complex, non-linear relationships between features (e.g., area, number of rooms, amenities) and property prices, which are common in real estate markets.
2. **Robustness to outliers**: It is less sensitive to outliers in the data compared to linear models, which is important given the presence of high-value luxury properties.
3. **Feature importance**: Random Forest provides a natural way to evaluate the relative importance of each feature in determining prices, helping to interpret results and support investment decisions.
4. **No need for heavy preprocessing**: It does not require extensive normalization or scaling of numerical features. Also they do not require as much pre-processing as other models.
5. **Proven performance**: Ensemble tree methods as a Random Forest are widely used in real estate price prediction for their reliability and accuracy.


We train two independent Random Forest models: one for predicting sales prices and another for rental prices. Using these predictions:
- For **sales**, we compute the difference between predicted and actual prices (*price gap*) to identify **underpriced properties**.
- For **rentals**, we estimate potential annual income and calculate the **gross rental yield (%)**, allowing us to identify districts with higher investment returns.


"""

In [ ]:
df["transaction_type"].value_counts()

In [ ]:
# Separate the dataset into sales and rentals
df_sales = df[df["transaction_type"].isin(["buy", "venta"])].copy()
df_rentals = df[df["transaction_type"].isin(["rent", "alquiler"])].copy()

print(f"Sales dataset shape: {df_sales.shape}")
print(f"Rentals dataset shape: {df_rentals.shape}")


In [ ]:
# Prepare the sales dataset
features = [
    "n_rooms", "n_baths", "area", "floor", "has_elevator", "has_terrace", "is_exterior",
    "has_swimming_pool", "has_parking", "has_garden", "has_energy_certificate",
    "has_floorplan", "is_discounted", "is_luxury"
]
df_sales_model = df_sales[features + ["district", "price"]].copy()
df_sales_model = pd.get_dummies(df_sales_model, columns=["district"], drop_first=True)

X_sales = df_sales_model.drop(columns=["price"])
y_sales = df_sales_model["price"]

# Split data
from sklearn.model_selection import train_test_split
X_train_s, X_test_s, y_train_s, y_test_s = train_test_split(X_sales, y_sales, test_size=0.2, random_state=42)

# Train Random Forest model
from sklearn.ensemble import RandomForestRegressor
model_sales = RandomForestRegressor(n_estimators=100, random_state=42, n_jobs=-1)
model_sales.fit(X_train_s, y_train_s)

# Evaluate
from sklearn.metrics import mean_absolute_error, r2_score
y_pred_s = model_sales.predict(X_test_s)
print(f"Sales Model - MAE: {mean_absolute_error(y_test_s, y_pred_s):,.0f} €")
print(f"Sales Model - R²: {r2_score(y_test_s, y_pred_s):.2f}")

# Predict prices and compute price gaps
df_sales["predicted_price"] = model_sales.predict(X_sales)
df_sales["price_gap"] = df_sales["predicted_price"] - df_sales["price"]
df_sales["undervalued"] = df_sales["price_gap"] > 50_000




In [ ]:
# Prepare the rentals dataset
df_rentals_model = df_rentals[features + ["district", "price"]].copy()
df_rentals_model = pd.get_dummies(df_rentals_model, columns=["district"], drop_first=True)

X_rentals = df_rentals_model.drop(columns=["price"])
y_rentals = df_rentals_model["price"]

# Split data
X_train_r, X_test_r, y_train_r, y_test_r = train_test_split(X_rentals, y_rentals, test_size=0.2, random_state=42)

# Train Random Forest model
model_rentals = RandomForestRegressor(n_estimators=100, random_state=42, n_jobs=-1)
model_rentals.fit(X_train_r, y_train_r)

# Evaluate
y_pred_r = model_rentals.predict(X_test_r)
print(f"Rentals Model - MAE: {mean_absolute_error(y_test_r, y_pred_r):,.0f} €")
print(f"Rentals Model - R²: {r2_score(y_test_r, y_pred_r):.2f}")

# Predict prices and compute price gaps
df_rentals["predicted_price"] = model_rentals.predict(X_rentals)
df_rentals["price_gap"] = df_rentals["predicted_price"] - df_rentals["price"]
df_rentals["undervalued"] = df_rentals["price_gap"] > 200  # Gap threshold for rentals (e.g. €200)


In [ ]:
def plot_feature_importances(model, X, title, ax=None, color=None):
    importances = pd.Series(model.feature_importances_, index=X.columns).sort_values(ascending=False)
    if ax is None:
        fig, ax = plt.subplots(figsize=(10, 6))
    importances.head(15).plot(kind="barh", ax=ax, color=color)
    ax.set_title(title)
    ax.set_xlabel("Importance")
    ax.invert_yaxis()
    return importances

fig, axes = plt.subplots(1, 2, figsize=(15, 6))
# Sales
sales_importances = plot_feature_importances(model_sales, X_sales, "Top 15 Features for Sales Price Prediction", ax = axes[0], color='lightblue')

# Rents
rental_importances = plot_feature_importances(model_rentals, X_rentals, "Top 15 Features for Rental Price Prediction", ax =axes[1], color='lightgreen')
plt.tight_layout()
plt.show()


In [ ]:
# Calculate gross rental yield (%)
avg_rent_district = df_rentals.groupby("district")["price"].mean().rename("avg_rent_price")
df_sales = df_sales.merge(avg_rent_district, on="district", how="left")
df_sales["gross_yield_pct"] = (df_sales["avg_rent_price"] * 12) / df_sales["price"] * 100

# Ranking: Districts with most undervalued properties
district_opps_sales = df_sales[df_sales["undervalued"]].groupby("district").agg(
    n_undervalued=("undervalued", "sum"),
    avg_price_gap=("price_gap", "mean"),
    avg_price=("price", "mean"),
    avg_price_m2=("price_per_m2", "mean"),
    avg_gross_yield_pct=("gross_yield_pct", "mean")
).sort_values(by="n_undervalued", ascending=False)

print("\nTop 15 districts with most undervalued sales properties:")
display(district_opps_sales.head(15))



In [ ]:
filtered_districts = district_opps_sales[district_opps_sales.index != "Unknown"].head(10).reset_index()

plt.figure(figsize=(10, 6))
scatter = plt.scatter(
    filtered_districts["avg_price_gap"],
    filtered_districts["avg_gross_yield_pct"],
    s=filtered_districts["n_undervalued"]*0.5,  
    c=filtered_districts["avg_gross_yield_pct"], 
    cmap="coolwarm", alpha=0.7, edgecolors="k"
)

for i, txt in enumerate(filtered_districts["district"]):
    plt.annotate(txt, (filtered_districts["avg_price_gap"][i], filtered_districts["avg_gross_yield_pct"][i]), fontsize=9, weight='bold')


plt.title("Undervalued Sales Properties: District Highlights", fontsize=14, weight="bold")
plt.xlabel("Average Price Gap (€)")
plt.ylabel("Gross Yield (%)")
plt.colorbar(scatter, label="Gross Yield (%)")
plt.grid(True)
plt.tight_layout()



In [ ]:

# Ranking: Districts with most undervalued rental properties
district_opps_rentals = df_rentals[df_rentals["undervalued"]].groupby("district").agg(
    n_undervalued=("undervalued", "sum"),
    avg_price_gap=("price_gap", "mean"),
    avg_price=("price", "mean"),
    avg_price_m2=("price_per_m2", "mean")
).sort_values(by="n_undervalued", ascending=False)

print("\nTop 15 districts with most undervalued rental properties:")
display(district_opps_rentals.head(15))


In [ ]:

filtered_rentals = district_opps_rentals[district_opps_rentals.index != "Unknown"].head(10).reset_index()

plt.figure(figsize=(10, 6))
scatter = plt.scatter(
    filtered_rentals["avg_price_gap"],
    filtered_rentals["avg_price_m2"],
    s=filtered_rentals["n_undervalued"]*0.5,  
    c=filtered_rentals["avg_price_m2"],  
    cmap="viridis", alpha=0.7, edgecolors="k"
)

for i, txt in enumerate(filtered_rentals["district"]):
    plt.annotate(txt, (filtered_rentals["avg_price_gap"][i], filtered_rentals["avg_price_m2"][i]), fontsize=9, weight='bold')


plt.title("Undervalued Rental Properties: District Highlights", fontsize=14, weight="bold")
plt.xlabel("Average Price Gap (€)")
plt.ylabel("Average Price per m² (€)")
plt.colorbar(scatter, label="Average Price per m² (€)")
plt.grid(True)
plt.tight_layout()


In [ ]:

# Scatter plot: real vs predicted prices (sales)
plt.figure(figsize=(8, 6))
plt.scatter(y_test_s, y_pred_s, alpha=0.4, edgecolors="k")
plt.plot([y_test_s.min(), y_test_s.max()], [y_test_s.min(), y_test_s.max()], 'r--', lw=2)
plt.title("Sales Model: Actual vs Predicted Prices", fontsize=14, weight="bold")
plt.xlabel("Actual Price (€)")
plt.ylabel("Predicted Price (€)")
plt.gca().xaxis.set_major_formatter(plt.FuncFormatter(lambda x, _: f"€{int(x):,}"))
plt.grid(True)
plt.tight_layout()

